# Typedict

TypedDict는 Python 3.8에서 도입된 기능으로, 딕셔너리의 각 키에 대해 특정 타입을 지정할 수 있게 해줍니다. 이를 통해 코드의 가독성과 유지보수성을 높이고, 정적 타입 검사 도구(예: mypy)를 활용하여 타입 오류를 사전에 방지할 수 있습니다.  

주요 특징:
- 키-값 타입 지정: 딕셔너리의 각 키에 대해 예상되는 값의 타입을 명시적으로 정의할 수 있습니다.  
- 정적 타입 검사 지원: TypedDict로 정의된 구조는 정적 타입 검사 도구와 함께 사용되어, 코드 작성 시 타입 불일치를 사전에 감지할 수 있습니다.   
- 선택적 및 필수 키 구분: 필요에 따라 특정 키를 필수 또는 선택적으로 지정할 수 있습니다.

In [44]:
from typing import TypedDict

class Movie(TypedDict):
    id: int        # 제품의 고유 식별자
    name: str      # 제품명
    price: float   # 제품 가격

# Movie라는 타입을 가진 movie 변수를 정의하고, 해당 변수에 영화 정보를 담은 딕셔너리를 할당
movie: Movie = {
    'id': 1,
    'name': 'Inception',
    'price': 19.99
}

# Pydantic

Pydantic은 Python에서 데이터 유효성 검사와 설정 관리를 위한 라이브러리로, 타입 힌트를 활용하여 데이터 모델을 정의하고 검증하는 데 중점을 둡니다.

### 1. 기본 모델 정의 및 데이터 검증
Pydantic을 사용하면 데이터 모델을 정의하고, 입력 데이터의 유효성을 자동으로 검사할 수 있습니다.

예) Product 모델은 id, name, price 필드를 가지며, 각각 정수, 문자열, 부동소수점 숫자로 정의되어 있습니다.

In [7]:
from pydantic import BaseModel

class Product(BaseModel):
    id: int        # 제품의 고유 식별자
    name: str      # 제품명
    price: float   # 제품 가격

In [11]:
# 유효한 데이터
product = Product(id=1, name="Laptop", price=999.99)
print(product)

id=1 name='Laptop' price=999.99


In [12]:
# 유효하지 않은 데이터
try:
    invalid_product = Product(id="one", name="Laptop", price="cheap")
except ValueError as e:
    print(e)

2 validation errors for Product
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='one', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/int_parsing
price
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='cheap', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/float_parsing


### 2. 선택적 필드와 기본값
Pydantic을 사용하면 선택적 필드와 기본값을 쉽게 정의할 수 있습니다.  

다음 예에서 department 필드는 선택 사항이며 기본값은 None입니다. is_active 필드는 기본값이 True로 설정되어 있습니다.

In [13]:
from typing import Optional
from pydantic import BaseModel

class Employee(BaseModel):
    id: int    # 직원의 고유 식별자
    name: str  # 직원 이름
    department: Optional[str] = None  # 부서명 (선택 사항)
    is_active: bool = True            # 현재 근무 상태 (기본값: True)

# Employee 인스턴스 생성
employee = Employee(id=1, name="Alice")
print(employee)

id=1 name='Alice' department=None is_active=True


### 3. 중첩 모델
Pydantic은 다른 모델을 필드로 포함하여 복잡한 데이터 구조를 표현할 수 있습니다.  

다음 코드에서 User 모델은 여러 개의 Address를 가질 수 있으며, 이를 통해 복잡한 데이터 구조를 표현합니다.

In [14]:
from typing import List  
from pydantic import BaseModel  

# 주소 정보를 나타내는 모델
class Address(BaseModel):
    street: str  # 거리 이름
    city: str    # 도시 이름
    country: str  # 국가 이름

# 사용자 정보를 나타내는 모델
class User(BaseModel):
    id: int       # 사용자의 고유 식별자
    name: str     # 사용자 이름
    addresses: List[Address]  # 사용자의 주소 목록 (Address 객체의 리스트)

# User 인스턴스 생성
user = User(
    id=1,            # 사용자 ID
    name="Charlie",  # 사용자 이름
    addresses=[       # 주소 목록
        Address(street="123 Main St", city="Seoul", country="South Korea"),
        Address(street="456 Side St", city="New York", country="USA") 
    ]
)

print(user)

id=1 name='Charlie' addresses=[Address(street='123 Main St', city='Seoul', country='South Korea'), Address(street='456 Side St', city='New York', country='USA')]


In [20]:
# # User 인스턴스 생성
# user = User(
#     id='abc',            # 사용자 ID
#     name=123,  # 사용자 이름
#     addresses=[       # 주소 목록
#         Address(street="123 Main St", city="Seoul", country="South Korea"),
#         Address(street="456 Side St", city="New York", country="USA") 
#     ]
# )

### 4. 데이터 직렬화 및 역직렬화
Pydantic 모델은 JSON과 같은 형식으로 쉽게 변환할 수 있으며, 그 반대도 가능합니다.  

다음 코드에서 Item 모델은 딕셔너리나 JSON 문자열로 변환될 수 있으며, 이를 통해 데이터베이스 저장, 네트워크 전송 등을 쉽게 처리할 수 있습니다.

In [23]:
from pydantic import BaseModel  

# 상품 정보를 나타내는 데이터 모델
class Item(BaseModel):
    name: str     # 상품 이름
    price: float  # 상품 가격

# Item 인스턴스 생성
item = Item(name="Book", price=12.99) 

# 모델을 딕셔너리로 변환
item_dict = item.model_dump()  
print(item_dict)  

# 모델을 JSON 문자열로 변환
item_json = item.model_dump_json()  
print(item_json) 

# 딕셔너리로부터 모델 생성
new_item = Item(**item_dict)  # 딕셔너리의 값을 언패킹하여 새로운 Item 객체 생성
print(new_item) 

{'name': 'Book', 'price': 12.99}
{"name":"Book","price":12.99}
name='Book' price=12.99


### 5. 커스텀 유효성 검사
Pydantic은 `@field_validator` 데코레이터를 사용하여 필드에 대한 추가적인 유효성 검사를 정의할 수 있습니다.  

다음 코드에서 check_age 메서드는 age 필드가 0에서 150 사이의 값인지 확인하며, 그렇지 않으면 ValueError를 발생시킵니다.

In [32]:
from pydantic import BaseModel, field_validator  

# 사용자 정보를 나타내는 데이터 모델
class User(BaseModel):
    id: int       # 사용자 고유 식별자
    name: str     # 사용자 이름
    age: int      # 사용자 나이

    # 나이(age) 필드에 대한 유효성 검사
    @field_validator('age')
    def check_age(cls, v):
        """
        나이는 0에서 150 사이여야 합니다.
        """
        if v < 0 or v > 150:
            raise ValueError('Age는 0 ~ 150 사이라야 합니다.')  # 유효하지 않은 경우 오류 발생
        return v  # 유효한 경우 나이 반환

# 유효한 나이로 User 객체 생성
user = User(id=1, name="Dave", age=30)  
print(user) 
print()

# 유효하지 않은 나이로 User 객체 생성 시 예외 처리
try:
    invalid_user = User(id=2, name="Eve", age=200) 
except ValueError as e:
    print(e) 

id=1 name='Dave' age=30

1 validation error for User
age
  Value error, Age는 0 ~ 150 사이라야 합니다. [type=value_error, input_value=200, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error


# Enum

열거형이라고도 하며, 서로 연관된 상수들의 집합을 정의하는 데 사용되는 특별한 데이터 타입입니다. 이를 통해 변수는 미리 정의된 값들 중 하나만을 가질 수 있게 제한되며, 코드의 가독성과 안정성을 높이는 데 기여합니다.

### 주요 특징:
- 타입 안전성 보장: 열거형을 사용하면 변수에 허용되지 않은 값이 할당되는 것을 방지하여, 코드의 안정성을 높일 수 있습니다.  
- 가독성 향상: 관련된 상수들을 그룹화하여 의미를 명확히 표현함으로써, 코드의 가독성을 향상시킵니다.  
- 상수 그룹화: 관련된 상수들을 하나의 열거형으로 묶어 관리할 수 있습니다.

In [33]:
from enum import Enum

class Day(Enum):
    SUNDAY = 0
    MONDAY = 1
    TUESDAY = 2
    WEDNESDAY = 3
    THURSDAY = 4
    FRIDAY = 5
    SATURDAY = 6

today = Day.MONDAY
today

<Day.MONDAY: 1>

# Pydantic 과 Enum 을 결합한 예제

In [39]:
from enum import Enum
from pydantic import BaseModel, Field, ValidationError

# 사용자 역할을 정의하는 열거형(Enum)
class Role(Enum):
    ADMIN = "admin"    # 관리자 역할: 모든 권한을 가짐
    EDITOR = "editor"  # 편집자 역할: 일부 콘텐츠를 수정할 수 있음
    VIEWER = "viewer"  # 뷰어 역할: 콘텐츠를 보기만 할 수 있음

# 사용자 모델 정의
class User(BaseModel):
    id: int = Field(..., description="사용자 고유 ID")   # 필수 필드 (Mandatory Field)
    name: str = Field(..., description="사용자 이름")
    role: Role = Field(..., description="사용자 역할 (admin, editor, viewer 중 하나)")
    age: int = Field(..., ge=0, le=150, description="나이 (0-150 사이)")

    def is_admin(self) -> bool:
        """사용자가 ADMIN 역할인지 확인합니다."""
        return self.role == Role.ADMIN

In [41]:
# 유효한 사용자 데이터 예제
try:
    user = User(
        id=1,
        name="Alice",
        role=Role.ADMIN,
        age=30
    )
    print("유효한 사용자 생성 성공!")
    print(user)
    print("Is Admin:", user.is_admin())
except Exception as e:
    print("유효하지 않은 사용자 데이터입니다.")
    print(e)

유효한 사용자 생성 성공!
id=1 name='Alice' role=<Role.ADMIN: 'admin'> age=30
Is Admin: True


In [42]:
# 유효하지 않은 사용자 데이터 예제 (잘못된 role 값)
try:
    invalid_user = User(
        id=2,
        name="Bob",
        role="superuser",  # Enum에 정의되지 않은 값
        age=25
    )
except Exception as e:
    print("유효하지 않은 사용자 데이터입니다 (role 오류).")
    print(e)

유효하지 않은 사용자 데이터입니다 (role 오류).
1 validation error for User
role
  Input should be 'admin', 'editor' or 'viewer' [type=enum, input_value='superuser', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/enum


In [43]:
# 유효하지 않은 사용자 데이터 예제 (나이 범위 초과)
try:
    invalid_user = User(
        id=3,
        name="Charlie",
        role=Role.VIEWER,
        age=200  # 범위를 벗어난 나이
    )
except Exception as e:
    print("유효하지 않은 사용자 데이터입니다 (age 오류).")
    print(e)

유효하지 않은 사용자 데이터입니다 (age 오류).
1 validation error for User
age
  Input should be less than or equal to 150 [type=less_than_equal, input_value=200, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/less_than_equal
